In [5]:
# Jeśli jeszcze nie zainstalowany:
pip install pydub

SyntaxError: invalid syntax (<ipython-input-5-31c80fb1fcfe>, line 2)

In [1]:
import wave
from pydub import AudioSegment as auds
import pandas as pd
import numpy as np
import os, fnmatch
from matplotlib import pyplot as plt
import scipy.io.wavfile as wf
from scipy.integrate import simps

C:\Users\Lenovo\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
path = 'C:/<WLASNA_SCIEZKA>/Nagrania'

# Wyciaganie nazw plikow ze sciezki
list_of_files = os.listdir(path)
pattern = "*.txt"
list_of_names = []
for entry in list_of_files:
    if fnmatch.fnmatch(entry, pattern):
            list_of_names.append(entry[:-4])

# Utworzenie list plikow wav i txt
files_wav_list = []
files_txt_list = []
for filename in list_of_names:
    files_wav_list.append(filename + '.wav')
    files_txt_list.append(filename + '.txt')
    
print('Obecne pliki:')

for i in range(len(list_of_names)):
    print(i, files_txt_list[i], files_wav_list[i])
print('liczba plikow: ', len(list_of_names))

Obecne pliki:
0 289516_25_K_01_2.txt 289516_25_K_01_2.wav
1 289516_25_K_10_3.txt 289516_25_K_10_3.wav
2 289516_25_K_16_4.txt 289516_25_K_16_4.wav
3 289516_25_K_22_1.txt 289516_25_K_22_1.wav
4 289540_23_K_13_2.txt 289540_23_K_13_2.wav
5 289540_23_K_18_3.txt 289540_23_K_18_3.wav
6 289540_23_K_22_4.txt 289540_23_K_22_4.wav
7 289540_23_K_9_1.txt 289540_23_K_9_1.wav
8 289550_23_K_18_2.txt 289550_23_K_18_2.wav
9 289550_23_K_23_3.txt 289550_23_K_23_3.wav
10 289550_23_K_23_4.txt 289550_23_K_23_4.wav
11 289550_23_K_9_1.txt 289550_23_K_9_1.wav
12 289556_24_K_11_1.txt 289556_24_K_11_1.wav
13 289556_24_K_15_2.txt 289556_24_K_15_2.wav
14 289556_24_K_19_3.txt 289556_24_K_19_3.wav
15 289556_24_K_23_4.txt 289556_24_K_23_4.wav
16 289592_24_K_12_1.txt 289592_24_K_12_1.wav
17 289592_24_K_15_2.txt 289592_24_K_15_2.wav
18 289592_24_K_19_3.txt 289592_24_K_19_3.wav
19 289592_24_K_7_4.txt 289592_24_K_7_4.wav
20 404907_23_K_11_1.txt 404907_23_K_11_1.wav
21 404907_23_K_14_2.txt 404907_23_K_14_2.wav
22 404907_23

In [38]:
# Liczenie parametrów zapisywanych do plików słów (w każdym pliku 24 wartości, kolejne osoby wypowiadające to samo słowo)
# Parametry:
# area (pole powierzchni z wartości bezwzględnych amplitud)
# zeros (gęstość przejść przez 0)
# squaresum (suma kwadratów amplitud)
# mean (średnia arytmetyczna amplitud)

area_folder_name = path + '/' + 'Area_słowa' + '/' # tworzenie folderu na wyniki danego parametru
zeros_folder_name = path + '/' + 'Zeros_słowa' + '/'
squaresum_folder_name = path + '/' + 'Squaresum_słowa' + '/'
mean_folder_name = path + '/' + 'Mean_słowa' + '/'
for j in range(98):
    print(j)
    labels = []
    if j > 0:
        for a in range(0, j):
            labels.append('')
    area_word = []
    zeros_word = []
    zeros_cnt = 0
    squaresum_word = []
    squaresum_cnt = 0
    squaresum_m = 0
    mean_word = []
    mean_cnt = 0    
    slowo = []
    for i in range(0, len(list_of_names)):
        filename = list_of_names[i]
        file_wav = files_wav_list[i]
        file_txt = files_txt_list[i]
        textfile = pd.read_csv(path + '/' + file_txt, sep = '\t', header = None)
        labels.append(str(textfile[2][j]))
        if labels[j] == 'nan':
            labels[j] = 'NA'
        labels[j] = labels[j].upper()
        if i == 0:
            print(labels[j])
        person_path = path + '/' + labels[j]
        # Sprawdź, czy istnieje plik o danej nazwie
        if os.path.isfile(person_path + '/' + filename + '_' + '{0}.wav'.format(labels[j])) == True:
            new_filename = filename + '_' + labels[j] + '.wav'
            person = wave.open(person_path + '/' + new_filename, 'rb')
            # Jeśli ilość ramek nieparzysta, to nie zadziała, więc wtedy wczytujemy wszystkie ramki oprócz ostatniej
            if person.getnframes() % 2 != 0:
                person_frames = person.readframes(person.getnframes()-1)
            else:
                person_frames = person.readframes(person.getnframes())
            person_frames = np.frombuffer(person_frames, dtype=int)
            slowo.append(person_frames)
            if os.path.isdir(area_folder_name) == False:
                os.mkdir(area_folder_name)
            if os.path.isdir(zeros_folder_name) == False:
                os.mkdir(zeros_folder_name)
            if os.path.isdir(squaresum_folder_name) == False:
                os.mkdir(squaresum_folder_name)
            if os.path.isdir(mean_folder_name) == False:
                os.mkdir(mean_folder_name)
            area_word.append(simps(np.abs(slowo[i])))
            for m in range(1, len(slowo[i])):
                if (slowo[i][m] <= 0 and slowo[i][m-1] > 0) or (slowo[i][m] >= 0 and slowo[i][m-1] < 0):
                    zeros_cnt += 1
            zeros_word.append(zeros_cnt) #liczone zera
            for n in range(0, len(slowo[i])):
                squaresum_m = int(slowo[i][n])*int(slowo[i][n])
                squaresum_cnt = squaresum_cnt + squaresum_m
            squaresum_word.append(squaresum_cnt)
            for o in range(0, len(slowo[i])):
                mean_cnt = mean_cnt + int(slowo[i][o])
            mean_cnt = mean_cnt/len(slowo[i])
            mean_word.append(mean_cnt)
                
    area_txt_person_filename = area_folder_name + 'area_' + labels[j] + '.txt'
    zeros_txt_person_filename = zeros_folder_name + 'zeros_' + labels[j] + '.txt'
    squaresum_txt_person_filename = squaresum_folder_name + 'squaresum_' + labels[j] + '.txt'
    mean_txt_person_filename = mean_folder_name + 'mean_' + labels[j] + '.txt'
    if os.path.isfile(area_txt_person_filename) == False:
        np.savetxt(area_txt_person_filename, area_word)
        print('Zapisano jako:', area_txt_person_filename)
    else:
        print('Plik', labels[j], 'z AREA już istnieje') 
    if os.path.isfile(zeros_txt_person_filename) == False:
        np.savetxt(zeros_txt_person_filename, zeros_word)
        print('Zapisano jako:', zeros_txt_person_filename)
    elif os.path.isfile(zeros_txt_person_filename) == True:
        print('Plik', labels[j], 'z ZEROS już istnieje') 
    if os.path.isfile(squaresum_txt_person_filename) == False:
        np.savetxt(squaresum_txt_person_filename, squaresum_word)
        print('Zapisano jako:', squaresum_txt_person_filename)
    elif os.path.isfile(squaresum_txt_person_filename) == True:
        print('Plik', labels[j], 'z SQUARESUM już istnieje')    
    if  os.path.isfile(mean_txt_person_filename) == False:
        np.savetxt(mean_txt_person_filename, mean_word)
        print('Zapisano jako:', mean_txt_person_filename)
    elif os.path.isfile(mean_txt_person_filename) == True:
        print('Plik', labels[j], 'z MEAN już istnieje')


0
OTWORZ
Plik OTWORZ z AREA już istnieje
Plik OTWORZ z ZEROS już istnieje
Zapisano jako: C:/Dysk D/STUDIA/Eksploracja danych biomedycznych/Nagrania/Squaresum_słowa/squaresum_OTWORZ.txt
Plik OTWORZ z MEAN już istnieje
1
WLACZ
Plik WLACZ z AREA już istnieje
Plik WLACZ z ZEROS już istnieje
Zapisano jako: C:/Dysk D/STUDIA/Eksploracja danych biomedycznych/Nagrania/Squaresum_słowa/squaresum_WLACZ.txt
Plik WLACZ z MEAN już istnieje
2
SWIATLO
Plik SWIATLO z AREA już istnieje
Plik SWIATLO z ZEROS już istnieje
Zapisano jako: C:/Dysk D/STUDIA/Eksploracja danych biomedycznych/Nagrania/Squaresum_słowa/squaresum_SWIATLO.txt
Plik SWIATLO z MEAN już istnieje
3
TELEWIZOR
Plik TELEWIZOR z AREA już istnieje
Plik TELEWIZOR z ZEROS już istnieje
Zapisano jako: C:/Dysk D/STUDIA/Eksploracja danych biomedycznych/Nagrania/Squaresum_słowa/squaresum_TELEWIZOR.txt
Plik TELEWIZOR z MEAN już istnieje
4
ZAMKNIJ
Plik ZAMKNIJ z AREA już istnieje
Plik ZAMKNIJ z ZEROS już istnieje
Zapisano jako: C:/Dysk D/STUDIA/Eksplora